# Assignment 3 by: HurHyeonBin (S3740878)


In [371]:
# Replace "YourFullName" with your full name; replace "YourStudentID" with your student ID (e.g. s1234567)

# Task 1: User-based Collaborative Filtering


In [372]:
# Start from here to write your code for Task 1. Please feel free to add more cells in this ipynb file.
# Don't forget about comments.


In [373]:
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score


In [374]:
# Read ratings data and movies data
names = ['user_id', 'item_id', 'rating', 'timestamp']
rating_data = pd.read_csv('ratings.dat', sep='::', names=names, engine='python')
ratings_df = pd.DataFrame(rating_data)
n_users = rating_data['user_id'].unique().shape[0]
n_items = rating_data['item_id'].unique().shape[0]

n_user_max = rating_data['user_id'].max()
n_item_max = rating_data['item_id'].max()

Since the ratings data not includes all movies information, and it couse the indexing error, the n_item_max will be used.

In [375]:
m_cols = ['item_id', 'title', 'genres']
movies_data = pd.read_csv('movies.dat', sep='::', names=m_cols, encoding='ISO-8859-1')
movies_data

<ipython-input-375-ea55cb41167a>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_data = pd.read_csv('movies.dat', sep='::', names=m_cols, encoding='ISO-8859-1')


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [376]:
# Create a DataFrame for all item id
all_movie_ids = pd.DataFrame({'item_id': range(1, n_item_max + 1)})
# Merge the all_movie_ids DataFrame with the original movie_data DataFrame
merged_movies_data = all_movie_ids.merge(movies_data, on='item_id', how='left')
merged_rating_data = all_movie_ids.merge(rating_data, on='item_id', how='left')
df = pd.merge(merged_rating_data, merged_movies_data, on='item_id', how='inner')

To match the index with the movies_data and rating_data, the all_movie_ids is used.

In [377]:
agg_ratings = df.groupby('item_id').agg(mean_rating = ('rating', 'mean'),
                                                number_of_ratings = ('rating', 'count')).reset_index()
df = pd.merge(df, agg_ratings[['item_id']], on='item_id', how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000455 entries, 0 to 1000454
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   item_id    1000455 non-null  int64  
 1   user_id    1000209 non-null  float64
 2   rating     1000209 non-null  float64
 3   timestamp  1000209 non-null  float64
 4   title      1000386 non-null  object 
 5   genres     1000386 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 53.4+ MB


In [378]:
# Create user_id - item_id table
matrix_user = df.pivot_table(index='user_id', columns='item_id', values='rating')
matrix_user.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [379]:
# Normalize the created user_id-item_id table
matrix_norm_user = matrix_user.subtract(matrix_user.mean(axis=1), axis = 'rows')
matrix_norm_user.head()


item_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,0.811321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,-1.146465,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [380]:
# create user_id - user_id table to check the similarity using Pearson correlation
user_similarity = matrix_norm_user.T.corr()
user_similarity.head()

user_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,6031.0,6032.0,6033.0,6034.0,6035.0,6036.0,6037.0,6038.0,6039.0,6040.0
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,1.000000,0.416667,-0.332182,0.333333,-0.172516,0.219956,NaN,-0.583333,3.305715e-01,-0.057363,...,-0.160128,7.559289e-01,1.000000,NaN,0.381246,-0.201132,-4.517540e-01,NaN,0.056857,-0.043519
2.0,0.416667,1.000000,0.236834,-0.118678,-0.208333,-0.061859,0.154303,0.024968,2.051123e-01,-0.091578,...,-0.709299,9.083405e-02,0.301309,NaN,0.390907,0.377852,3.081451e-01,-0.500000,0.572078,-0.027143
3.0,-0.332182,0.236834,1.000000,0.840168,-0.437621,-0.349215,0.790569,-0.528594,8.012345e-18,0.009305,...,-0.395285,-9.759001e-02,-0.711556,NaN,0.427001,0.204619,2.485765e-17,0.500000,0.309277,-0.395285
4.0,0.333333,-0.118678,0.840168,1.000000,0.000000,NaN,-0.207514,0.534522,5.222330e-01,-0.270501,...,-0.645497,-4.532467e-17,-0.078598,NaN,-0.013558,0.467447,-1.132277e-01,-1.000000,0.000000,0.450323
5.0,-0.172516,-0.208333,-0.437621,0.000000,1.000000,-0.592014,-0.028516,0.218130,-3.345621e-17,-0.023582,...,0.260875,7.071068e-01,0.116775,0.5,0.539112,0.160690,-9.713482e-02,-0.928571,1.000000,0.308273


In [381]:
# Since the last user_id is 6040, the randommly generated user is random user_id from 1 to 6040
import random
picked_userid = random.randint(1, 6040)
k_array = [5,10,15,20,25]

In [382]:
# Number of similar users
k = 5
picked_userid = random.randint(1, 6041)

k_array = [5,10,15,20,25]

for k in k_array:

# Find similar k number of user
  similar_users = user_similarity[picked_userid].sort_values(ascending=False)[:k]

# Find the movie that the similar users watched
  similar_user_movies = matrix_norm_user[matrix_norm_user.index.isin(similar_users.index)].dropna(axis=1, how='all')
  similar_user_movies

# Calculate the average rating value of the selected user
  selected_user_avg = matrix_user[matrix_user.index == picked_userid].T.mean()[picked_userid]

# Create an empty DataFrame to store item scores and predictions
  item_scores = pd.DataFrame(index=similar_user_movies.columns)

# Calculate the rating prediction, also add the average value of the selected user to make better accuracy.
  item_scores['predicted_rating'] = similar_user_movies.mean() + selected_user_avg
  print(item_scores)

# Print out the predicted ratings for all items
  print(f'Predicted ratings for user {picked_userid}:')

  actual_ratings = matrix_user[matrix_user.index == picked_userid]

# Calculate the difference between actual ratings and predicted ratings
  diff = actual_ratings.sub(item_scores['predicted_rating'], axis=1)

# Calculate the squared difference
  diff_squared = diff.pow(2)

# Calculate the mean squared difference
  mse = diff_squared.mean(axis=1)

# Calculate the RMSE
  rmse = np.sqrt(mse)

# Print the RMSE
  print(f'When k = {k}, the RMSE value is: {rmse.values[0]}')

         predicted_rating
item_id                  
1                4.326368
2                3.552806
3                3.382328
4                2.996841
5                3.308832
...                   ...
3948             3.908437
3949             4.576267
3950             3.913488
3951             4.103739
3952             4.051403

[3524 rows x 1 columns]
Predicted ratings for user 2898:
When k = 5, the RMSE value is: 0.6154439579838801
         predicted_rating
item_id                  
1                4.348702
2                3.514959
3                3.432569
4                3.124704
5                3.363767
...                   ...
3948             3.906930
3949             4.443972
3950             3.981735
3951             4.192333
3952             4.066136

[3657 rows x 1 columns]
Predicted ratings for user 2898:
When k = 10, the RMSE value is: 0.707633029874123
         predicted_rating
item_id                  
1                4.354821
2                3.520788
3   

It provides not omly the predicted rating for the movies from the selected user, but also provides the rmse values for each k value

# Task 2: Item-based Filtering

In [383]:
# Create item_id-user_id table
matrix_item = df.pivot_table(index='item_id', columns='user_id', values='rating')
matrix_item.head()

user_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,6031.0,6032.0,6033.0,6034.0,6035.0,6036.0,6037.0,6038.0,6039.0,6040.0
item_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [384]:
# Normalize the item_id-user_id table
matrix_norm_item = matrix_item.subtract(matrix_item.mean(axis=1), axis = 0)
matrix_norm_item.head()


user_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,6031.0,6032.0,6033.0,6034.0,6035.0,6036.0,6037.0,6038.0,6039.0,6040.0
item_id,,,,,,,,,,,,,,,,,,,,,
1,0.853154,NaN,NaN,NaN,NaN,-0.146846,NaN,-0.146846,0.853154,0.853154,...,NaN,-0.146846,NaN,NaN,-0.146846,NaN,NaN,NaN,NaN,-1.146846
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.798859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.016736,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.270588,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.729412,-0.729412,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.006757,NaN,NaN,NaN,NaN,NaN


In [385]:
# Create item-item similarity table using pearson corr
item_similarity = matrix_norm_item.T.corr()
item_similarity.head()


item_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
item_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.187467,0.160649,0.348759,0.156957,0.051097,0.149536,0.151248,-0.039358,0.143598,...,-0.099845,0.174078,0.146344,0.135146,0.431743,0.139323,0.189518,0.146536,3.243253e-01,0.157996
2,0.187467,1.000000,0.149349,0.057874,0.384293,0.075067,0.251093,0.312949,0.407857,0.254558,...,0.009798,-1.000000,0.276240,0.133538,0.042524,0.216196,0.082241,-0.154672,-3.535534e-01,0.073111
3,0.160649,0.149349,1.000000,0.261800,0.399532,0.156934,0.220211,0.394405,0.151163,0.269250,...,-0.134059,0.239046,-0.361158,0.205521,-0.024042,0.314249,0.026743,0.445399,NaN,0.110024
4,0.348759,0.057874,0.261800,1.000000,0.401954,-0.012096,0.097271,0.074080,-0.681895,0.010868,...,0.589662,NaN,NaN,0.158114,-0.500000,0.209258,-0.434783,0.500000,-9.614813e-17,0.154463
5,0.156957,0.384293,0.399532,0.401954,1.000000,0.075755,0.276160,0.589313,0.538139,0.241397,...,-0.533114,0.188982,NaN,0.522037,0.960769,0.416291,-0.133077,0.650000,NaN,-0.033736


In [386]:
# Item similarity matrix using cosine similarity
item_similarity_cosine = cosine_similarity(matrix_norm_item.fillna(0))
item_similarity_cosine

array([[ 1.        ,  0.07521343,  0.045759  , ...,  0.01329586,
         0.01160901,  0.02843184],
       [ 0.07521343,  1.        ,  0.03549382, ..., -0.0121648 ,
        -0.00640761,  0.00551753],
       [ 0.045759  ,  0.03549382,  1.        , ...,  0.02617915,
         0.01776199,  0.01474485],
       ...,
       [ 0.01329586, -0.0121648 ,  0.02617915, ...,  1.        ,
         0.03977157,  0.06668999],
       [ 0.01160901, -0.00640761,  0.01776199, ...,  0.03977157,
         1.        ,  0.11039995],
       [ 0.02843184,  0.00551753,  0.01474485, ...,  0.06668999,
         0.11039995,  1.        ]])

In [387]:
# pick the random movie using it's id
picked_movie = random.randint(1, 3000)
user_maxmatrix = df.pivot_table(index='user_id', columns='item_id', values='rating')
print(picked_movie)
results = pd.DataFrame(columns=['user_id', 'rating'])

for i in range(1, 6041):
    picked_userid = i
    # Movies that the target user has watched
    picked_userid_watched = pd.DataFrame(matrix_norm_item[picked_userid].dropna(axis=0, how='all') \
                                         .sort_values(ascending=False)) \
                                         .reset_index() \
                                         .rename(columns={1: 'rating'})

    picked_movie_similarity_score = item_similarity[[picked_movie]].reset_index().rename(columns={picked_movie: 'similarity_score'})
    k = 5
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched,
                                                right=picked_movie_similarity_score,
                                                on='item_id',
                                                how='inner') \
                                         .sort_values('similarity_score', ascending=False)[:k]

    predicted_rating = round(np.average(picked_userid_watched_similarity.iloc[:, 1], weights=picked_userid_watched_similarity.iloc[:, 2]), 6)
    avg_rating = user_maxmatrix[user_maxmatrix.index == picked_userid].T.mean()[picked_userid]
    row = pd.DataFrame({'user_id': [picked_userid], 'rating': [predicted_rating + avg_rating]})
    results = pd.concat([results, row], ignore_index=True)
# Load real data to calculate the rmse
    actual_ratings = matrix_item[matrix_item.index == picked_movie]

print(results)
diff = actual_ratings.sub(results['rating'], axis=1)
diff_squared = diff.pow(2)
mse = diff_squared.mean(axis=1)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse.iloc[0]}')
user_maxmatrix = df.pivot_table(index='user_id', columns='item_id', values='rating')

389
     user_id    rating
0          1  4.523242
1          2  4.141849
2          3  4.021887
3          4  4.646344
4          5  3.120140
...      ...       ...
6035    6036  2.314811
6036    6037  3.318367
6037    6038  4.384943
6038    6039  3.734163
6039    6040  3.463737

[6040 rows x 2 columns]
RMSE: 1.1485413704875942


In [388]:
item_similarity_cosine_df = pd.DataFrame(item_similarity_cosine, columns=matrix_norm_item.index, index=matrix_norm_item.index)

user_maxmatrix = df.pivot_table(index='user_id', columns='item_id', values='rating')

results = pd.DataFrame(columns=['user_id', 'rating'])

for i in range(1, 6041):
    picked_userid = i
    # Movies that the target user has watched
    picked_userid_watched = pd.DataFrame(matrix_norm_item[picked_userid].dropna(axis=0, how='all') \
                                         .sort_values(ascending=False)) \
                                         .reset_index() \
                                         .rename(columns={1: 'rating'})

    picked_movie_similarity_score = item_similarity_cosine_df[[picked_movie]].reset_index().rename(columns={picked_movie: 'similarity_score'})
    n = 5
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched,
                                                right=picked_movie_similarity_score,
                                                on='item_id',
                                                how='inner') \
                                         .sort_values('similarity_score', ascending=False)[:n]

    predicted_rating = round(np.average(picked_userid_watched_similarity.iloc[:, 1], weights=picked_userid_watched_similarity.iloc[:, 2]), 6)
    avg_rating = user_maxmatrix[user_maxmatrix.index == picked_userid].T.mean()[picked_userid]
    row = pd.DataFrame({'user_id': [picked_userid], 'rating': [predicted_rating + avg_rating]})
    results = pd.concat([results, row], ignore_index=True)

    actual_ratings = matrix_item[matrix_item.index == picked_movie]

# Print the first few rows of the results
print(results)
diff = actual_ratings.sub(results['rating'], axis=1)
diff_squared = diff.pow(2)
mse = diff_squared.mean(axis=1)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse.iloc[0]}')


     user_id    rating
0          1  4.816179
1          2  3.971160
2          3  4.122788
3          4  4.617355
4          5  2.917918
...      ...       ...
6035    6036  2.946013
6036    6037  3.219962
6037    6038  4.345501
6038    6039  3.652594
6039    6040  3.860135

[6040 rows x 2 columns]
RMSE: 0.8626807961782949


# Task 3: A Better Recommender System

In [389]:
# Task 3.1
#  compute result of the user_based and item based collaborative filtering



In [390]:
movies_data = pd.read_csv('movies.dat', sep='::', names=m_cols, encoding='ISO-8859-1',engine='python')
rating_data = pd.read_csv('ratings.dat', sep='::', names=names, engine='python')
rating_dat = pd.DataFrame(rating_data)
movies_data

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [391]:
# # User similarity matrix using Pearson correlation
# user_similarity = matrix_norm_user.T.corr()
# user_similarity.head()


In [392]:
ratings = np.zeros((n_user_max, n_item_max))
ratingsNum = np.zeros((n_user_max,n_item_max))

In [393]:
for row in rating_data.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    ratingsNum[row[1]-1, row[2]-1] = 1
print(ratings)
print(ratingsNum)

[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [394]:
# for test
picked_userid = 1


In [395]:
# information about selected user
user_ratings = ratings[picked_userid, :]
sorted_indices = np.argsort(user_ratings)[::-1]
# choose top 100 items
top_n_indices = sorted_indices[:100]

k = 10

  # Get top n similar users
similar_users = user_similarity[picked_userid].sort_values(ascending=False)[:k]

  # Remove movies that have been watched
picked_userid_watched = matrix_norm_user[matrix_norm_user.index == picked_userid].dropna(axis=1, how='all')
picked_userid_watched

  # Movies that similar users watched. Remove movies that none of the similar users have watched
similar_user_movies = matrix_norm_user[matrix_norm_user.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_movies

item_score = {}


for i in similar_user_movies.columns:
    # Get the ratings for movie i
  movie_rating = similar_user_movies[i]
  total = 0
  count = 0
  for u in similar_users.index:
      # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
        # Score is the sum of user similarity score multiply by the movie rating
      score = similar_users[u] * movie_rating[u]
        # Add the score to the total score for the movie so far
      total += score
        # Add 1 to the count
      count +=1
    # Calculate the average score
  item_score[i] = total / count

item_score = pd.DataFrame(item_score.items(), columns=['item_id', 'movie_score'])

  # Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

  # Select top m movies
m = 30
ranked_item_score = pd.merge(ranked_item_score, movies_data, on='item_id', how='inner')
movie_id = ranked_item_score['item_id']
title = ranked_item_score['title']
movie_title_df = pd.DataFrame({'item_id': movie_id, 'title': title})
# select 500 users because if the number of chosen data are too low, it is hard to compare with the item_based filtering result
knncf = movie_title_df.head(500)
knncf.reset_index(drop=True, inplace=True)
movie_id_array = knncf['item_id'].to_numpy()
print(knncf)



     item_id                                              title
0       1434                               Stranger, The (1994)
1       2129                       Saltmen of Tibet, The (1997)
2       3172                            Ulysses (Ulisse) (1954)
3       1780                   Ayn Rand: A Sense of Life (1997)
4       2061                            Full Tilt Boogie (1997)
..       ...                                                ...
495     3510                                   Frequency (2000)
496      497                      Much Ado About Nothing (1993)
497      955                            Bringing Up Baby (1938)
498     2788  And Now for Something Completely Different (1971)
499      932                      Affair to Remember, An (1957)

[500 rows x 2 columns]


In [396]:
# change it to descending order
knncf = movie_title_df.head(500)
knncf = knncf.iloc[::-1].reset_index(drop=True)
movie_id_array = knncf['item_id'].to_numpy()
print(knncf)

     item_id                                              title
0        932                      Affair to Remember, An (1957)
1       2788  And Now for Something Completely Different (1971)
2        955                            Bringing Up Baby (1938)
3        497                      Much Ado About Nothing (1993)
4       3510                                   Frequency (2000)
..       ...                                                ...
495     2061                            Full Tilt Boogie (1997)
496     1780                   Ayn Rand: A Sense of Life (1997)
497     3172                            Ulysses (Ulisse) (1954)
498     2129                       Saltmen of Tibet, The (1997)
499     1434                               Stranger, The (1994)

[500 rows x 2 columns]


In [397]:
# # Create user-item matrix
# matrix = df.pivot_table(index='item_id', columns='user_id', values='rating')
# matrix.head()

# # Normalize user-item matrix
# matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 0)
# matrix_norm.head()

# # Item similarity matrix using cosine similarity
# item_similarity_cosine = cosine_similarity(matrix_norm.fillna(0))
# item_similarity_cosine

In [398]:
item_similarity_cosine_df = pd.DataFrame(item_similarity_cosine, columns=matrix_norm_item.index, index=matrix_norm_item.index)

user_maxmatrix = df.pivot_table(index='user_id', columns='item_id', values='rating')

results = pd.DataFrame(columns=['user_id', 'rating'])

for i in range(1, 6041):
    picked_userid = i
    # Movies that the target user has watched
    picked_userid_watched = pd.DataFrame(matrix_norm_item[picked_userid].dropna(axis=0, how='all') \
                                         .sort_values(ascending=False)) \
                                         .reset_index() \
                                         .rename(columns={1: 'rating'})

    picked_movie_similarity_score = item_similarity_cosine_df[[picked_movie]].reset_index().rename(columns={picked_movie: 'similarity_score'})
    n = 5
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched,
                                                right=picked_movie_similarity_score,
                                                on='item_id',
                                                how='inner') \
                                         .sort_values('similarity_score', ascending=False)[:n]

    predicted_rating = round(np.average(picked_userid_watched_similarity.iloc[:, 1], weights=picked_userid_watched_similarity.iloc[:, 2]), 6)
    avg_rating = user_maxmatrix[user_maxmatrix.index == picked_userid].T.mean()[picked_userid]
    row = pd.DataFrame({'user_id': [picked_userid], 'rating': [predicted_rating + avg_rating]})
    results = pd.concat([results, row], ignore_index=True)

    actual_ratings = matrix_item[matrix_item.index == picked_movie]

# Print the first few rows of the results
print(results)
diff = actual_ratings.sub(results['rating'], axis=1)
diff_squared = diff.pow(2)
mse = diff_squared.mean(axis=1)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse.iloc[0]}')


     user_id    rating
0          1  4.816179
1          2  3.971160
2          3  4.122788
3          4  4.617355
4          5  2.917918
...      ...       ...
6035    6036  2.946013
6036    6037  3.219962
6037    6038  4.345501
6038    6039  3.652594
6039    6040  3.860135

[6040 rows x 2 columns]
RMSE: 0.8626807961782949


In [399]:
# Item-based recommendation function
def item_based_rec(picked_userid=50, number_of_similar_items=5, number_of_recommendations = 500):
  import operator
  # selected users are not watched yet
  picked_userid_unwatched = pd.DataFrame(matrix_norm_item[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['item_id'].values.tolist()
  print(picked_userid_unwatched)
  # selected users already watched
  picked_userid_watched = pd.DataFrame(matrix_norm_item[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={1:'rating'})

  rating_prediction ={}

  for picked_movie in picked_userid_unwatched:
    # Calculate the similarity score of the picked movie iwth other movies
    picked_movie_similarity_score = item_similarity[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched,
                                                right=picked_movie_similarity_score,
                                                on='item_id',
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items].rename(columns={picked_userid:'rating'})

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1

    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'],
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

# Get recommendations
recommended_movie = item_based_rec(picked_userid=1, number_of_similar_items=5, number_of_recommendations =500)
recommended_movie

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 23

[(470, 1.142228),
 (458, 1.099),
 (518, 1.018547),
 (15, 0.99537),
 (23, 0.971109),
 (542, 0.962689),
 (26, 0.934885),
 (74, 0.933437),
 (77, 0.923347),
 (468, 0.897279),
 (85, 0.896765),
 (79, 0.870929),
 (17, 0.856396),
 (36, 0.838458),
 (430, 0.834214),
 (119, 0.81695),
 (34, 0.815318),
 (481, 0.81497),
 (60, 0.81403),
 (477, 0.786801),
 (27, 0.778653),
 (453, 0.767461),
 (61, 0.763295),
 (5, 0.752128),
 (435, 0.751924),
 (39, 0.750941),
 (480, 0.749764),
 (16, 0.738739),
 (455, 0.735543),
 (107, 0.732559),
 (55, 0.72797),
 (70, 0.723289),
 (462, 0.719767),
 (512, 0.703757),
 (504, 0.700648),
 (507, 0.700406),
 (76, 0.6958),
 (503, 0.695729),
 (491, 0.694782),
 (428, 0.693621),
 (511, 0.693363),
 (43, 0.688574),
 (12, 0.688145),
 (7, 0.684003),
 (492, 0.682341),
 (500, 0.679858),
 (62, 0.67668),
 (68, 0.67492),
 (525, 0.674325),
 (101, 0.674253),
 (19, 0.665431),
 (42, 0.662678),
 (31, 0.657877),
 (22, 0.655354),
 (13, 0.651834),
 (526, 0.647694),
 (436, 0.642091),
 (24, 0.63915),
 

In [400]:
recommended_movie_df = pd.DataFrame(recommended_movie, columns=['item_id', 'score'])

recommended_movie_df = recommended_movie_df.merge(movies_data, on='item_id', how='left')

recommended_movie_df = recommended_movie_df[['item_id', 'title']]
print(recommended_movie_df)

     item_id                                title
0        470                 House Party 3 (1994)
1        458  Geronimo: An American Legend (1993)
2        518        Road to Wellville, The (1994)
3         15              Cutthroat Island (1995)
4         23                     Assassins (1995)
..       ...                                  ...
495      501                         Naked (1993)
496      505                         North (1994)
497      496          What Happened Was... (1994)
498      515       Remains of the Day, The (1993)
499      540                        Sliver (1993)

[500 rows x 2 columns]


In [401]:
# change it to descending order
recommended_movie_df = recommended_movie_df[['item_id', 'title']]
recommended_movie_df = recommended_movie_df.iloc[::-1].reset_index(drop=True)
print(recommended_movie_df)

     item_id                                title
0        540                        Sliver (1993)
1        515       Remains of the Day, The (1993)
2        496          What Happened Was... (1994)
3        505                         North (1994)
4        501                         Naked (1993)
..       ...                                  ...
495       23                     Assassins (1995)
496       15              Cutthroat Island (1995)
497      518        Road to Wellville, The (1994)
498      458  Geronimo: An American Legend (1993)
499      470                 House Party 3 (1994)

[500 rows x 2 columns]


In [402]:
print(recommended_movie_df)
print(knncf)

     item_id                                title
0        540                        Sliver (1993)
1        515       Remains of the Day, The (1993)
2        496          What Happened Was... (1994)
3        505                         North (1994)
4        501                         Naked (1993)
..       ...                                  ...
495       23                     Assassins (1995)
496       15              Cutthroat Island (1995)
497      518        Road to Wellville, The (1994)
498      458  Geronimo: An American Legend (1993)
499      470                 House Party 3 (1994)

[500 rows x 2 columns]
     item_id                                              title
0        932                      Affair to Remember, An (1957)
1       2788  And Now for Something Completely Different (1971)
2        955                            Bringing Up Baby (1938)
3        497                      Much Ado About Nothing (1993)
4       3510                                   Frequency

In [403]:
knncf = movie_title_df.head(500)
knncf.reset_index(drop=True, inplace=True)
knncf['point'] = knncf.index
movie_id_array = knncf['item_id'].to_numpy()
print(knncf)

     item_id                                              title  point
0       1434                               Stranger, The (1994)      0
1       2129                       Saltmen of Tibet, The (1997)      1
2       3172                            Ulysses (Ulisse) (1954)      2
3       1780                   Ayn Rand: A Sense of Life (1997)      3
4       2061                            Full Tilt Boogie (1997)      4
..       ...                                                ...    ...
495     3510                                   Frequency (2000)    495
496      497                      Much Ado About Nothing (1993)    496
497      955                            Bringing Up Baby (1938)    497
498     2788  And Now for Something Completely Different (1971)    498
499      932                      Affair to Remember, An (1957)    499

[500 rows x 3 columns]


<ipython-input-403-51eb1e0cce66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knncf['point'] = knncf.index


In [404]:
common_items = knncf[knncf['item_id'].isin(recommended_movie_df['item_id'])]

common_item_ids = common_items['item_id'].to_list()

# compare with the user_based result and item_based result
for item_id in common_item_ids:
    index_in_knncf = knncf[knncf['item_id'] == item_id].index[0]
    index_in_recommended = recommended_movie_df[recommended_movie_df['item_id'] == item_id].index[0]
    # if the item is in the item_based result, index will be added
    knncf.at[index_in_knncf, 'point'] += index_in_recommended
    # print overlap
    print(item_id)

print(knncf)

134
37
318
116
110
50
363
356
530
296
394
457
47
326
111
401
246
306
199
29
28
90
214
290
128
293
83
251
58
508
73
175
6
428
17
138
529
474
16
364
223
497
     item_id                                              title  point
0       1434                               Stranger, The (1994)      0
1       2129                       Saltmen of Tibet, The (1997)      1
2       3172                            Ulysses (Ulisse) (1954)      2
3       1780                   Ayn Rand: A Sense of Life (1997)      3
4       2061                            Full Tilt Boogie (1997)      4
..       ...                                                ...    ...
495     3510                                   Frequency (2000)    495
496      497                      Much Ado About Nothing (1993)    509
497      955                            Bringing Up Baby (1938)    497
498     2788  And Now for Something Completely Different (1971)    498
499      932                      Affair to Remember, An (1957) 

In [405]:
knncf = knncf.sort_values(by='point', ascending=False)
print(knncf)

     item_id                                            title  point
476       16                                    Casino (1995)    948
442       17                     Sense and Sensibility (1995)    929
440      428                             Bronx Tale, A (1993)    900
475      474                       In the Line of Fire (1993)    879
370       83  Once Upon a Time... When We Were Colored (1995)    797
..       ...                                              ...    ...
4       2061                          Full Tilt Boogie (1997)      4
3       1780                 Ayn Rand: A Sense of Life (1997)      3
2       3172                          Ulysses (Ulisse) (1954)      2
1       2129                     Saltmen of Tibet, The (1997)      1
0       1434                             Stranger, The (1994)      0

[500 rows x 3 columns]


In [406]:
Option2RecSys = knncf.head(30)
# print out the final recommend movies
print(Option2RecSys)

     item_id                                              title  point
476       16                                      Casino (1995)    948
442       17                       Sense and Sensibility (1995)    929
440      428                               Bronx Tale, A (1993)    900
475      474                         In the Line of Fire (1993)    879
370       83    Once Upon a Time... When We Were Colored (1995)    797
386       58                   Postino, Il (The Postman) (1994)    788
436        6                                        Heat (1995)    781
402       73                             Misérables, Les (1995)    763
447      138                             Neon Bible, The (1995)    751
303       90                 Journey of August King, The (1995)    716
420      175                                        Kids (1995)    683
278       28                                  Persuasion (1995)    678
484      223                                      Clerks (1994)    667
378   

In [407]:
# Task 3.2


In [408]:
def precision(recommended_items, relevant_items):
    # Calculate the intersection of recommended_items and relevant_items
    true_positive = len(set(recommended_items).intersection(set(relevant_items)))

    # Calculate the total number of recommended items
    total_recommended_items = len(recommended_items)

    # Calculate precision
    precision_value = true_positive / total_recommended_items if total_recommended_items > 0 else 0
    return precision_value

In [409]:
top_30_knncf = knncf.head(30)
item_id_array = top_30_knncf['item_id'].to_numpy()
print(item_id_array)

[  16   17  428  474   83   58    6   73  138   90  175   28  223  251
   29  128  457   47  214  364  497  111  290  932 2788  955 3510 3552
 1235 1534]


In [410]:
precision_value = precision(top_n_indices, item_id_array)
print(f"Precision: {precision_value:.2f}")

Precision: 0.00


In [411]:
#MovieAvg recommendation
movies_data = pd.read_csv('movies.dat', sep='::', names=m_cols, encoding='ISO-8859-1',engine='python')
rating_data = pd.read_csv('ratings.dat', sep='::', names=names, engine='python')
rating_dat = pd.DataFrame(rating_data)
movies_data

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [412]:
# rating_data

In [413]:
user_ratings_count = rating_data['user_id'].value_counts()
users_with_100_or_more_ratings = user_ratings_count[user_ratings_count >= 100].index.tolist()

users_with_100_or_more_ratings_array = np.array(users_with_100_or_more_ratings)

print(users_with_100_or_more_ratings_array)

[4169 1680 4277 ... 1481 1567 3206]


In [414]:
last_index = len(users_with_100_or_more_ratings_array) - 1

In [415]:
random_indices = random.sample(range(last_index + 1), 5)
print("5 random index :", random_indices)

5 random index : [1751, 890, 161, 1710, 1578]


In [416]:
selected_user_ids = [users_with_100_or_more_ratings_array[i] for i in random_indices]

print("selected user", selected_user_ids)

선택된 사용자 ID: [1415, 1591, 509, 654, 4114]


In [417]:
print(last_index)

2944


In [418]:
n_users = rating_data.user_id.unique().shape[0]
n_items = rating_data.item_id.unique().shape[0]
print(str(n_users)+ 'users')
print(str(n_items)+'items')

n_user_max = rating_data['user_id'].max()
n_item_max = rating_data['item_id'].max()

print(n_user_max)
print(n_item_max)

6040users
3706items
6040
3952


In [419]:
ratings = np.zeros((n_user_max, n_item_max))
ratingsNum = np.zeros((n_user_max,n_item_max))

In [420]:
for row in rating_data.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    ratingsNum[row[1]-1, row[2]-1] = 1
print(ratings)
print(ratingsNum)

[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [421]:
itemRateNum = ratingsNum.sum(axis=0)
itemRateSum = ratings.sum(axis=0)

print(itemRateNum)
print(itemRateSum)

[2077.  701.  478. ...   54.   40.  388.]
[8613. 2244. 1442. ...  198.  156. 1467.]


In [422]:
itemRateAvg = itemRateSum / itemRateNum
print(itemRateAvg)

[4.14684641 3.20114123 3.0167364  ... 3.66666667 3.9        3.78092784]


<ipython-input-422-4d598cf96026>:1: RuntimeWarning: invalid value encountered in divide
  itemRateAvg = itemRateSum / itemRateNum


In [423]:
from re import I
for u in selected_user_ids:
  top_n = 30
  activeUser = u

  user_ratings = ratings[u, :]
  sorted_indices = np.argsort(user_ratings)[::-1]
  top_n_indices = sorted_indices[:500]


  mask_activeUser = ratings[activeUser, :]>0
  itemRateAvgCurrent  = itemRateAvg.copy()
  itemRateAvgCurrent[mask_activeUser] = 0
  itemSortInd = itemRateAvgCurrent.argsort()

  print('movie Id' + '\t movie title')
  MovieAvg = movies_data['title'][itemSortInd[list(range(len(itemSortInd) -1, len(itemSortInd)-top_n -1, -1))]]

  MovieAvg_predictedData = MovieAvg.index.to_numpy()
  print(MovieAvg)


  top_n_indices = np.array(top_n_indices)
  MovieAvg_predictedData = np.array(MovieAvg_predictedData)
  # compare with the 500 real data and 30 recommand data
  precision_value = precision(top_n_indices, MovieAvg_predictedData)
  print(f"Precision: {precision_value:.2f}")

movie Id	 movie title
815                                      Diebinnen (1995)
722                             Heaven's Prisoners (1996)
1491                            Fifth Element, The (1997)
682                              Mrs. Winterbourne (1996)
685                                Substitute, The (1996)
688                              Feeling Minnesota (1996)
692                                          Daens (1992)
698                               Mulholland Falls (1996)
1490                                  Fathers' Day (1997)
1480                                       Volcano (1997)
712              Halfmoon (Paul Bowles - Halbmond) (1995)
1477                                A Chef in Love (1996)
1468                           Grosse Pointe Blank (1997)
1064                  Monty Python's Life of Brian (1979)
720     Institute Benjamenta, or This Dream People Cal...
1466                         Inventing the Abbotts (1997)
726            Cemetery Man (Dellamorte Dellamore)

In [424]:
matrix = df.pivot_table(index='user_id', columns='item_id', values='rating')
matrix.head()
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')
matrix_norm.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,0.811321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,-1.146465,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [425]:
for u_id in selected_user_ids:
  picked_userid = u_id

  k = 5
  user_ratings = ratings[u_id, :]
  sorted_indices = np.argsort(user_ratings)[::-1]
  top_n_indices = sorted_indices[:500]

  # Get top n similar users
  similar_users = user_similarity[picked_userid].sort_values(ascending=False)[:k]

  # Remove movies that have been watched
  picked_userid_watched = matrix_norm[matrix_norm.index == picked_userid].dropna(axis=1, how='all')
  picked_userid_watched

  # Movies that similar users watched. Remove movies that none of the similar users have watched
  similar_user_movies = matrix_norm[matrix_norm.index.isin(similar_users.index)].dropna(axis=1, how='all')
  similar_user_movies

  item_score = {}

  # Loop through items
  for i in similar_user_movies.columns:
    # Get the ratings for movie i
    movie_rating = similar_user_movies[i]
    # Create a variable to store the score
    total = 0
    # Create a variable to store the number of scores
    count = 0
    # Loop through similar users
    for u in similar_users.index:
      # If the movie has rating
      if pd.isna(movie_rating[u]) == False:
        # Score is the sum of user similarity score multiply by the movie rating
        score = similar_users[u] * movie_rating[u]
        # Add the score to the total score for the movie so far
        total += score
        # Add 1 to the count
        count +=1
    # Get the average score for the item
    item_score[i] = total / count

  # Convert dictionary to pandas dataframe
  item_score = pd.DataFrame(item_score.items(), columns=['item_id', 'movie_score'])

  # Sort the movies by score
  ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

  # Select top m movies
  m = 30
  ranked_item_score = pd.merge(ranked_item_score, movies_data, on='item_id', how='inner')
  movie_id = ranked_item_score['item_id']
  title = ranked_item_score['title']
  movie_title_df = pd.DataFrame({'movie_id': movie_id, 'title': title})

  knncf = movie_title_df.head(30)
  movie_id_array = knncf['movie_id'].to_numpy()
  print(knncf)
  precision_value = precision(movie_id_array, top_n_indices)
  print(f"Precision: {precision_value:.2f}")


    movie_id                                              title
0        989          Schlafes Bruder (Brother of Sleep) (1995)
1       2342                              Hard Core Logo (1996)
2       2931        Time of the Gypsies (Dom za vesanje) (1989)
3        864                                   Wife, The (1995)
4       3232                               Seven Chances (1925)
5        557                                  Mamma Roma (1962)
6       3607                           One Little Indian (1973)
7       3245                I Am Cuba (Soy Cuba/Ya Kuba) (1964)
8       3522       Sacco and Vanzetti (Sacco e Vanzetti) (1971)
9         53                                    Lamerica (1994)
10       214              Before the Rain (Pred dozhdot) (1994)
11      1434                               Stranger, The (1994)
12       787                 Gate of Heavenly Peace, The (1995)
13      2858                             American Beauty (1999)
14      3905                            

In [426]:
for u_id in selected_user_ids:
  picked_user_id = u_id
  user_ratings = ratings[picked_userid, :]
  sorted_indices = np.argsort(user_ratings)[::-1]
  top_n_indices = sorted_indices[:100]

  k = 10

  # Get top n similar users
  similar_users = user_similarity[picked_userid].sort_values(ascending=False)[:k]

  # Remove movies that have been watched
  picked_userid_watched = matrix_norm_user[matrix_norm_user.index == picked_userid].dropna(axis=1, how='all')
  picked_userid_watched

  # Movies that similar users watched. Remove movies that none of the similar users have watched
  similar_user_movies = matrix_norm_user[matrix_norm_user.index.isin(similar_users.index)].dropna(axis=1, how='all')
  similar_user_movies

  item_score = {}

  # Loop through items
  for i in similar_user_movies.columns:
    # Get the ratings for movie i
    movie_rating = similar_user_movies[i]
    # Create a variable to store the score
    total = 0
    # Create a variable to store the number of scores
    count = 0
    # Loop through similar users
    for u in similar_users.index:
      # If the movie has rating
      if pd.isna(movie_rating[u]) == False:
        # Score is the sum of user similarity score multiply by the movie rating
        score = similar_users[u] * movie_rating[u]
        # Add the score to the total score for the movie so far
        total += score
        # Add 1 to the count
        count +=1
    # Get the average score for the item
    item_score[i] = total / count

  # Convert dictionary to pandas dataframe
  item_score = pd.DataFrame(item_score.items(), columns=['item_id', 'movie_score'])

  # Sort the movies by score
  ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

  ranked_item_score = pd.merge(ranked_item_score, movies_data, on='item_id', how='inner')
  movie_id = ranked_item_score['item_id']
  title = ranked_item_score['title']
  movie_title_df = pd.DataFrame({'item_id': movie_id, 'title': title})

  knncf = movie_title_df.head(500)
  knncf.reset_index(drop=True, inplace=True)

  recommended_movie = item_based_rec(picked_userid=u_id, number_of_similar_items=5, number_of_recommendations = 500)

  recommended_movie_df = pd.DataFrame(recommended_movie, columns=['item_id', 'score'])

# movies_data 데이터프레임에서 movie_id에 해당하는 영화 제목을 가져와서 병합
  recommended_movie_df = recommended_movie_df.merge(movies_data, on='item_id', how='left')

  movie_id_array = knncf['item_id'].to_numpy()
  knncf.reset_index(drop=True, inplace=True)
  knncf['point'] = knncf.index
  movie_id_array = knncf['item_id'].to_numpy()


# 결과를 출력
  recommended_movie_df = recommended_movie_df[['item_id', 'title']]
  recommended_movie_df = recommended_movie_df.iloc[::-1].reset_index(drop=True)
  common_items = knncf[knncf['item_id'].isin(recommended_movie_df['item_id'])]

# common_items의 'item_id' 값을 추출하여 리스트로 변환
  common_item_ids = common_items['item_id'].to_list()

# knncf 데이터프레임에서 'point' 열을 업데이트
  for item_id in common_item_ids:
      index_in_knncf = knncf[knncf['item_id'] == item_id].index[0]  # item_id에 해당하는 인덱스
      index_in_recommended = recommended_movie_df[recommended_movie_df['item_id'] == item_id].index[0]  # recommended_movie_df에서의 인덱스
      knncf.at[index_in_knncf, 'point'] += index_in_recommended

  knncf = knncf.sort_values(by='point', ascending=False)
  Option2RecSys = knncf.head(30)
  item_id_array = Option2RecSys['item_id'].to_numpy()
  precision_value = precision(top_n_indices, item_id_array)
  print(Option2RecSys)
  print(f"Precision: {precision_value:.2f}")


[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 111, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 230, 233, 234, 235, 

<ipython-input-426-f4cb938e5839>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knncf['point'] = knncf.index


     item_id                                           title  point
496       17                    Sense and Sensibility (1995)    978
468      480                            Jurassic Park (1993)    877
402      497                   Much Ado About Nothing (1993)    829
408       58                Postino, Il (The Postman) (1994)    744
452      176                       Living in Oblivion (1995)    738
465      125                   Flirting With Disaster (1996)    731
384        6                                     Heat (1995)    715
378       16                                   Casino (1995)    701
396      123                        Chungking Express (1994)    672
336      101                            Bottle Rocket (1996)    666
182       53                                 Lamerica (1994)    661
255       32                           Twelve Monkeys (1995)    655
187      428                            Bronx Tale, A (1993)    652
411      213  Burnt By the Sun (Utomlyonnye soln

<ipython-input-426-f4cb938e5839>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knncf['point'] = knncf.index


     item_id                                           title  point
496       17                    Sense and Sensibility (1995)    953
449        1                                Toy Story (1995)    904
408       58                Postino, Il (The Postman) (1994)    875
378       16                                   Casino (1995)    838
384        6                                     Heat (1995)    835
465      125                   Flirting With Disaster (1996)    816
396      123                        Chungking Express (1994)    767
386      162                                    Crumb (1994)    759
452      176                       Living in Oblivion (1995)    757
336      101                            Bottle Rocket (1996)    750
411      213  Burnt By the Sun (Utomlyonnye solntsem) (1994)    727
209       97                         Hate (Haine, La) (1995)    704
255       32                           Twelve Monkeys (1995)    680
188       28                               Persu

<ipython-input-426-f4cb938e5839>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knncf['point'] = knncf.index


     item_id                                           title  point
471     2189               I Married A Strange Person (1997)    889
396      123                        Chungking Express (1994)    888
378       16                                   Casino (1995)    844
496       17                    Sense and Sensibility (1995)    837
407     2197                                Firelight (1997)    822
376     2203                        Shadow of a Doubt (1943)    802
353     2395                                 Rushmore (1998)    784
429     2109                                Jerk, The (1979)    777
465      125                   Flirting With Disaster (1996)    747
296     2186                     Strangers on a Train (1951)    721
452      176                       Living in Oblivion (1995)    706
328     2351  Nights of Cabiria (Le Notti di Cabiria) (1957)    694
386      162                                    Crumb (1994)    680
182       53                                 Lam

<ipython-input-426-f4cb938e5839>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knncf['point'] = knncf.index


     item_id                                           title  point
496       17                    Sense and Sensibility (1995)    901
402      497                   Much Ado About Nothing (1993)    867
449        1                                Toy Story (1995)    853
408       58                Postino, Il (The Postman) (1994)    847
481      475                In the Name of the Father (1993)    776
336      101                            Bottle Rocket (1996)    757
465      125                   Flirting With Disaster (1996)    730
452      176                       Living in Oblivion (1995)    719
396      123                        Chungking Express (1994)    665
386      162                                    Crumb (1994)    642
151      953                    It's a Wonderful Life (1946)    630
184       29               City of Lost Children, The (1995)    630
188       28                               Persuasion (1995)    623
187      428                            Bronx Ta

<ipython-input-426-f4cb938e5839>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knncf['point'] = knncf.index


In [427]:
# Item-based recommendation function
def item_based_rec(picked_userid=50, number_of_similar_items=5, number_of_recommendations = 500):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_norm_item[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['item_id'].values.tolist()
  print(picked_userid_unwatched)
  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_norm_item[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={1:'rating'})

  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}

  # Loop through unwatched movies
  for picked_movie in picked_userid_unwatched:
    # Calculate the similarity score of the picked movie iwth other movies
    picked_movie_similarity_score = item_similarity[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched,
                                                right=picked_movie_similarity_score,
                                                on='item_id',
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items].rename(columns={picked_userid:'rating'})

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1

    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'],
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

# Get recommendations
recommended_movie = item_based_rec(picked_userid=1, number_of_similar_items=5, number_of_recommendations =500)
recommended_movie

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 23

[(470, 1.142228),
 (458, 1.099),
 (518, 1.018547),
 (15, 0.99537),
 (23, 0.971109),
 (542, 0.962689),
 (26, 0.934885),
 (74, 0.933437),
 (77, 0.923347),
 (468, 0.897279),
 (85, 0.896765),
 (79, 0.870929),
 (17, 0.856396),
 (36, 0.838458),
 (430, 0.834214),
 (119, 0.81695),
 (34, 0.815318),
 (481, 0.81497),
 (60, 0.81403),
 (477, 0.786801),
 (27, 0.778653),
 (453, 0.767461),
 (61, 0.763295),
 (5, 0.752128),
 (435, 0.751924),
 (39, 0.750941),
 (480, 0.749764),
 (16, 0.738739),
 (455, 0.735543),
 (107, 0.732559),
 (55, 0.72797),
 (70, 0.723289),
 (462, 0.719767),
 (512, 0.703757),
 (504, 0.700648),
 (507, 0.700406),
 (76, 0.6958),
 (503, 0.695729),
 (491, 0.694782),
 (428, 0.693621),
 (511, 0.693363),
 (43, 0.688574),
 (12, 0.688145),
 (7, 0.684003),
 (492, 0.682341),
 (500, 0.679858),
 (62, 0.67668),
 (68, 0.67492),
 (525, 0.674325),
 (101, 0.674253),
 (19, 0.665431),
 (42, 0.662678),
 (31, 0.657877),
 (22, 0.655354),
 (13, 0.651834),
 (526, 0.647694),
 (436, 0.642091),
 (24, 0.63915),
 

In [428]:
recommended_movie = item_based_rec(picked_userid=3, number_of_similar_items=5, number_of_recommendations = 500)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 